In [210]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import myslack
import datetime as dt
import pickle
import time
import re
from tqdm import trange

In [225]:
driver = webdriver.Chrome()  
url = 'https://www.yogiyo.co.kr/mobile/?utm_source=google&utm_medium=cpc&utm_campaign=sem_bra_orignal&utm_term=%EC%9A%94%EA%B8%B0%EC%9A%94&utm_id=sem_000001&referrer=adjust_tracker%3Dgrm2aa%26adjust_google_network%3Dg%26adjust_google_placement%3D%26adjust_campaign%3Dsem_bra_web_kr_700886066_cpc%26adjust_adgroup%3D38249737818%26adjust_creative%3D%EC%9A%94%EA%B8%B0%EC%9A%94_b&gclid=EAIaIQobChMIv9DaxIL-3AIVA6yWCh0C8glmEAAYASAAEgLbF_D_BwE#/%EC%84%9C%EC%9A%B8/138223/'
driver.get(url) 

In [226]:
food_dict = {
    '프랜차이즈':3, '치킨':4, '피자':5, '양식':5, '중국집':6,
    '한식':7, '일식':8, '돈가스':8, '족발':9, '보쌈':9, 
    '야식':10, '분식':11, '카페':12, '디저트':12 }

In [227]:
# 현재 위치로 설정하기
def set_location():
    driver.find_element_by_xpath('//*[@id="search"]/div/span[1]/button').click()
    print('현재 위치로 설정하는중...')
    time.sleep(5)
    print('현재 위치 설정 완료!')
    
# 카테고리 페이지로 넘어가기
def go_to_category(category):
    driver.find_element_by_xpath('//*[@id="category"]/ul/li[{}]/span'.format(food_dict.get(category))).click()

# 해당 카테고리의 음식점 갯수 저장
def get_restaurant_count():
    return int(driver.find_element_by_xpath('//*[@id="restaurant_count"]').text)    
    
# 해당 카테고리의 음식점 페이지로 넘어가기    
def go_to_restaurant(restaurantnum):
    driver.find_element_by_xpath('//*[@id="content"]/div/div[4]/div[{}]/div'.format(restaurantnum)).click()
    
# 해당 음식점의 리뷰 페이지로 넘어가기
def go_to_review():
    driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a').click()
    
# 해당 음식점의 리뷰 갯수 반환
def get_review_count():
    return int(driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a/span').text)
    
# 페이지 한번 맨아래로 내리기
def scroll_bottom():
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")

# 카테고리(음식점 리스트) 페이지에서 음식점 리스트 로드하기
def cat_page_scroll_repeat(restaurant_count):
    # 브라우저의 스크롤 위치 조절
    height=200+118*(restaurant_count//2)
    driver.execute_script("window.scrollTo(0,{})".format(height))
    print('카테고리(음식점 리스트) 페이지 (0,{}) 위치로 내리는중...'.format(height))
    time.sleep(3)
    
# 더보기 클릭하기 
def click_more_review():
    driver.find_element_by_class_name('btn-more').click()    

# 리뷰 페이지 모두 펼치기
def stretch_review_page():
    review_count = int(driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a/span').text)
    click_count = int((review_count/10))
    print('모든 리뷰 불러오는중...')
    for _ in trange(click_count):
        try:
            scroll_bottom()
            print('리뷰 스크롤 내리는중...')
            click_more_review()
            time.sleep(1)
        except Exception as e:
            pass
    print('모든 리뷰 불러오기 성공!')
        
# 페이지 뒤로 가기 (한 음식점 리뷰를 모두 모았으면 다시 음식점 리스트 페이지로 돌아감)
def go_back_page():
    driver.execute_script("window.history.go(-1)")    

In [228]:
def yogiyo_crawling(category):
    set_location()
    go_to_category(category) # 해당 카테고리(음식점 리스트) 페이지로 넘어감 
    print(category+'페이지로 넘어가는 중...')
    time.sleep(2)
    
    df = pd.DataFrame(columns=['Restaurant','UserID','Menu','Review',
                               'Taste','Quantity','Delivery','Date'])
    
    print('Start {} Crawling...'.format(category))
    for i in trange(get_restaurant_count()): # 해당 카테고리의 음식점 개수만큼 돌아감
        cat_page_scroll_repeat(i) # i번째 음식점 위치로 page를 내림
        
        go_to_restaurant(i+2) # 순서대로 각 음식점 페이지로 넘어감
        print(str(i+1)+'번째 음식점 페이지로 넘어가는중...')
        time.sleep(2)
        
        go_to_review() # 해당 음식점의 리뷰페이지로 넘어감
        print(str(i+1)+'번째 음식점 리뷰 페이지로 넘어가는중...')
        time.sleep(2)
        
        stretch_review_page() # 해당 음식점의 모든 리뷰를 불러옴

        for j in trange(get_review_count()):  # 해당 음식점의 리뷰 수 만큼 데이터를 가져옴
            try:
                df.loc[len(df)] = { 
                    'Restaurant':driver.find_element_by_class_name('restaurant-name').text,
                    'UserID':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/div[1]/span[1]'.format(j+2)).text,
                    'Menu':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/div[3]'.format(j+2)).text,
                    'Review':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/p'.format(j+2)).text,
                    'Taste':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/div[2]/div/span[2]/span[3]'.format(j+2)).text,
                    'Quantity':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/div[2]/div/span[2]/span[6]'.format(j+2)).text,
                    'Delivery':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/div[2]/div/span[2]/span[9]'.format(j+2)).text,
                    'Date':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/div[1]/span[2]'.format(j+2)).text,
                }
            except Exception as e:
                print('리뷰 페이지 에러')
                print(e)
                pass

        go_back_page() # 해당 음식점 리뷰를 모두 모았으면 다시 음식점 리스트 페이지로 돌아감
        print('음식점 리스트 페이지로 돌아가는중...')
        time.sleep(2)
                
    print('Finish {} Crawling!!!'.format(category))
    return df

In [ ]:
# for i in range(get_restaurant_count()):
#     cat_page_scroll_repeat(i)
#     driver.find_element_by_xpath('//*[@id="content"]/div/div[4]/div[{}]/div'.format(i+2)).click()
#     time.sleep(1)
#     print(str(i+1)+'번째 음식점 {} 방문'.format(driver.find_element_by_class_name('restaurant-name').text))
#     go_back_page()
#     time.sleep(2)

In [ ]:
%time chicken_df = yogiyo_crawling('치킨')

현재 위치로 설정하는중...


 55%|█████▌    | 273/496 [00:29<00:23,  9.36it/s]

현재 위치 설정 완료!
치킨페이지로 넘어가는 중...



  0%|          | 0/77 [00:00<?, ?it/s]

Start 치킨 Crawling...
카테고리(음식점 리스트) 페이지 (0,200) 위치로 내리는중...
1번째 음식점 페이지로 넘어가는중...


...

리뷰 스크롤 내리는중...



100%|██████████| 34/34 [00:36<00:00,  1.06s/it]

  0%|          | 0/340 [00:00<?, ?it/s]

모든 리뷰 불러오기 성공!



 46%|████▌     | 156/340 [00:39<00:46,  3.93it/s]


 85%|████████▌ | 290/340 [01:18<00:13,  3.71it/s]

In [ ]:
chicken_df.tail()

In [ ]:
myslack.send_slack('finish!')

In [49]:
driver.close() # 창닫기
driver.quit() # 브라우져 닫기

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.39.562713 (dd642283e958a93ebf6891600db055f1f1b4f3b2),platform=Mac OS X 10.13.6 x86_64)


---

In [358]:
# def yogiyo_crawling(category):
#     go_to_category(category) # 해당 카테고리(음식점 리스트) 페이지로 넘어감 
#     print(category+'페이지로 넘어가는 중...')
#     time.sleep(2)
    
#     df = pd.DataFrame(columns=['Restaurant','UserID','Menu','Review',
#                                'Taste','Quantity','Delivery','Date'])
    
#     print('Start {} Crawling...'.format(category))
#     for i in trange(get_store_count()): # 해당 카테고리의 음식점 개수만큼 돌아감
#         try:
#             scroll_bottom()
#             time.sleep(2)
#             go_to_store(i+2) # 순서대로 각 음식점 페이지로 넘어감
#             print(str(i+1)+'번째 음식점 페이지로 넘어가는중...')
#             time.sleep(2)
#             go_to_review() # 해당 음식점의 리뷰페이지로 넘어감
#             print('해당 음식점 리뷰 페이지로 넘어가는중...')
#             time.sleep(2)
#             stretch_review_page() # 해당 음식점의 모든 리뷰를 불러옴
            
#             for j in trange(get_review_count()):  # 해당 음식점의 리뷰 수 만큼 데이터를 가져옴
#                 try:
#                     df.loc[len(df)] = { 
#                         'Restaurant':driver.find_element_by_class_name('restaurant-name').text,
#                         'UserID':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/div[1]/span[1]'.format(j+2)).text,
#                         'Menu':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/div[3]'.format(j+2)).text,
#                         'Review':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/p'.format(j+2)).text,
#                         'Taste':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/div[2]/div/span[2]/span[3]'.format(j+2)).text,
#                         'Quantity':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/div[2]/div/span[2]/span[6]'.format(j+2)).text,
#                         'Delivery':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/div[2]/div/span[2]/span[9]'.format(j+2)).text,
#                         'Date':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/div[1]/span[2]'.format(j+2)).text,
#                     }
#                 except Exception as e:
#                     print('리뷰 페이지 에러')
#                     print(e)
#                     pass
                
#             go_back_page() # 해당 음식점 리뷰를 모두 모았으면 다시 음식점 리스트 페이지로 돌아감
#             print('음식점 리스트 페이지로 돌아가는중...')
#             time.sleep(2)
#             scroll_bottom()
#             time.sleep(2)
            
#         except Exception as e:
#             print('음식점 페이지 에러')
#             scroll_bottom()
#             print(e)
#             pass
                
#     print('Finish {} Crawling!!!'.format(category))
#     return df